In [ ]:
import json
from docx import Document
import PyPDF2
from openai import OpenAI


In [4]:
def callDeepseek(system_prompt, user_prompt):
    client = OpenAI(api_key="sk-a20fe5cabaac4bcda4af0347d3ad5038", base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
      ],
        max_tokens=4096,
        stream=False
    )

    return response.choices[0].message.content

def callDeepseekCoT(system_prompt, user_prompt):
    client = OpenAI(api_key="sk-a20fe5cabaac4bcda4af0347d3ad5038", base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
      ],
        max_tokens=4096,
        stream=False
    )

    return response.choices[0].message.reasoning_content
    
def callDeepseekWithJson(system_prompt, user_prompt):
    client = OpenAI(api_key="sk-a20fe5cabaac4bcda4af0347d3ad5038", base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt + "\nHere are some thoughts: " + callDeepseekCoT(system_prompt, user_prompt)},
      ],
        max_tokens=4096,
        response_format= {'type': 'json_object'},
        stream=False
    )
    
    return response.choices[0].message.content

In [5]:
class code:
    def __init__(self, input_code, fileType):
        if fileType == "text":
            self.input_code = input_code
            
        elif fileType == "docs":
            doc = Document("input_code")
            for paragraph in doc.paragraphs:
                self.input_code += paragraph
                
        elif fileType == "pdf":
            with open("input_code", "rb") as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    self.input_code += page

            
    def checkSyntaxError(self):
        system_prompt = """
        The user will provide some python code. Detect syntax errors such as indentation issues, missing colons, or unmatched brackets.
        You need to identify all syntax errors, and find out which line each of them is in, and output advice to correct for each line of the problemetic code.
        The output format should be a json object.
        
        EXAMPLE INPUT:
        1:def multiply(a, b)  
        2:    result = a * b
        3:    return result
        4:
        5:print(multiply(2, 3)  
        6:
        7:if 10 > 5  
        8:    print("True")
        
        EXAMPLE JSON OUTPUT:
        {
            "checks": [
                {
                    "errorType" : "syntax error",
                    "range" : [1],
                    "Advice": "Added : after the function definition."
                },
                {
                    "errorType" : "syntax error",
                    "range" : [5],
                    "Advice": "Completed the parentheses with )."
                },
                {
                    "errorType" : "syntax error",
                    "range" : [7],
                    "Advice": "Added : after the if condition."
                }
            ]
        }
        """
        user_prompt = self.input_code
        
        return callDeepseekWithJson(system_prompt, user_prompt)

    def checkCommonRuntimeError(self):
        system_prompt = """
        The user will provide some python code. Detect runtime errors such as NameError, TypeError, ZeroDivisionError, and explain why they occur.
        You need to identify all runtime errors, and find out which line each of them is in, and output advice to correct for each line of the problemetic code.
        The output format should be a json object.
        
        EXAMPLE INPUT:
        def calculate_average(numbers):
            total = sum(numbers)
            average = total / len(numbers)
            return average
        
        def print_list_element(lst):
            print(lst[5])
        
        def process_data(data):
            result = data + 10
            return result
        
        def main():
            print(calculate_average([1, 2, 3]))  
        
            my_list = [1, 2, 3]
            print_list_element(my_list)  
        
            process_data("abc")  
        
            misspelled_variable = 5
            print(misspeled_variable)
        
        if __name__ == "__main__":
            main()
        
        EXAMPLE JSON OUTPUT:
        {
            "checks": [
                {
                    "errorType" : "ZeroDivisionError",
                    "range" : [3],
                    "Advice": "Added : Add a guard clause to handle empty lists."
                },
                {
                    "errorType" : "IndexError",
                    "range" : [7],
                    "Advice": "Check list length or use exception handling."
                },
                {
                    "errorType" : "TypeError",
                    "range" : [10],
                    "Advice": "Explicitly convert data to an integer or handle type mismatch."
                },
                {
                    "errorType" : "NameError",
                    "range" : [19],
                    "Advice": "Correct the variable name spelling."
                }
            ]
        }
        """
        user_prompt = self.input_code
        
        return callDeepseek(system_prompt, user_prompt)

    def AlBasedLogicErrorDetection(self):
        system_prompt = """
        The user will provide some python code. Catch logic errors even when the code runs but produces incorrect results.
        """
        user_prompt = self.input_code
        
        return callDeepseek(system_prompt, user_prompt)

    def lineByLineAIExplanation(self):
        system_prompt = """
        The user will provide some python code. Explain each line of user-submitted code in plain English to help with debugging and learning.
        """
        user_prompt = self.input_code
        
        return callDeepseek(system_prompt, user_prompt)

In [6]:
code1 = code("""
def calculate_stats(data):
    total = sum(data)
    average = total / len(data)
    return {"total": total, "avg": average}

def process_user(user):
    user["full_name"] = user["name"] + " " + user["surname"]
    user["age_next_year"] = user.age + 1
    return user

def load_config():
    config = {"timeout": 30, "retries": 3}
    return config["retry_count"]

def main():
    stats = calculate_stats([])
    print(stats)

    user = {"name": "Alice"}
    processed_user = process_user(user)
    print(processed_user)

    config_value = load_config()
    print(config_value)

    value = "123"
    result = value + 5
    print(result)

if __name__ == "__main__":
    main()""",
            "text")


print(json.loads(code1.checkCommonRuntimeError()))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)